In [2]:
import requests
url = "http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

In [4]:
res = requests.get(url,stream = True)
with open('aclImdb_v1.tar.gz',"wb") as f:
    for chunk in res.iter_content(chunk_size=1024):
        if chunk:
            f.write(chunk)

In [6]:
import tarfile
tfile = tarfile.open("aclImdb_v1.tar.gz",'r:gz')
result = tfile.extractall()

In [1]:
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer

/home/shihao-z/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [14]:
import re
def rm_tags(text):
    re_tag = re.compile(r'<[^>]+>')
    return re_tag.sub('',text)

In [20]:
import os 
import progressbar
p = progressbar.ProgressBar()
def read_files(filetype):
    path = 'aclImdb/'
    file_list = []
    
    positive_path = path + filetype + '/pos/'
    for f in os.listdir(positive_path):
        file_list += [positive_path + f]
    
    negative_path = path + filetype + '/neg/'
    for f in os.listdir(negative_path):
        file_list += [negative_path + f]
    
    print('read',filetype,'files:',len(file_list))
    
    all_labels = ([1]*12500 + [0]*12500)
    
    all_texts = []
    
    for fi in p(file_list):
        with open(fi,encoding = 'utf8') as file_input:
            all_texts += [rm_tags(" ".join(file_input.readlines()))]
    
    return all_labels,all_texts

In [21]:
y_train,train_text = read_files('train')

read train files: 25000


100% |########################################################################|


In [19]:
y_test,test_text = read_files("test")

read test files: 25000


In [24]:
len(y_train)

25000

In [27]:
train_text[0]

"i went to see this movie with a bunch of friends one night. I didn't really hear much about it. So I wasn't expecting anything. But after I saw it, I really liked it. Nicolas Cage and the rest of the cast were very good. But I do have to say Giovanni Ribisi's acting performace did need a little perking up. But such a small flaw, it could be overrided. Gone In 60 Seconds is about a retired car thief who must boost 60 rare and exotic cars in one night to save his brother's life. The movie is in no way predictable. So the ending should be a suprise. Think it's just another, fast car driving movie? Well you are partially right. There is much more to it. Everyone should take a look at this movie."

In [78]:
token = Tokenizer(num_words = 4000)
token.fit_on_texts(train_text)

In [79]:
print(token.document_count)

25000


In [80]:
token.word_index

{'the': 1,
 'and': 2,
 'a': 3,
 'of': 4,
 'to': 5,
 'is': 6,
 'in': 7,
 'it': 8,
 'i': 9,
 'this': 10,
 'that': 11,
 'was': 12,
 'as': 13,
 'for': 14,
 'with': 15,
 'movie': 16,
 'but': 17,
 'film': 18,
 'on': 19,
 'not': 20,
 'you': 21,
 'are': 22,
 'his': 23,
 'have': 24,
 'be': 25,
 'he': 26,
 'one': 27,
 'all': 28,
 'at': 29,
 'by': 30,
 'an': 31,
 'they': 32,
 'who': 33,
 'so': 34,
 'from': 35,
 'like': 36,
 'her': 37,
 'or': 38,
 'just': 39,
 'about': 40,
 "it's": 41,
 'out': 42,
 'has': 43,
 'if': 44,
 'some': 45,
 'there': 46,
 'what': 47,
 'good': 48,
 'more': 49,
 'when': 50,
 'very': 51,
 'up': 52,
 'no': 53,
 'time': 54,
 'she': 55,
 'even': 56,
 'my': 57,
 'would': 58,
 'which': 59,
 'only': 60,
 'story': 61,
 'really': 62,
 'see': 63,
 'their': 64,
 'had': 65,
 'can': 66,
 'were': 67,
 'me': 68,
 'well': 69,
 'than': 70,
 'we': 71,
 'much': 72,
 'been': 73,
 'get': 74,
 'bad': 75,
 'will': 76,
 'also': 77,
 'do': 78,
 'into': 79,
 'people': 80,
 'other': 81,
 'first': 82,

In [34]:
x_train_seq = token.texts_to_sequences(train_text)
x_test_seq = token.texts_to_sequences(test_text)

In [35]:
train_text[0]

"i went to see this movie with a bunch of friends one night. I didn't really hear much about it. So I wasn't expecting anything. But after I saw it, I really liked it. Nicolas Cage and the rest of the cast were very good. But I do have to say Giovanni Ribisi's acting performace did need a little perking up. But such a small flaw, it could be overrided. Gone In 60 Seconds is about a retired car thief who must boost 60 rare and exotic cars in one night to save his brother's life. The movie is in no way predictable. So the ending should be a suprise. Think it's just another, fast car driving movie? Well you are partially right. There is much more to it. Everyone should take a look at this movie."

In [39]:
print(x_train_seq[0],len(x_train_seq[0]))

[9, 430, 5, 63, 10, 16, 15, 3, 756, 4, 365, 27, 310, 9, 156, 62, 838, 72, 40, 8, 34, 9, 281, 1012, 228, 17, 99, 9, 215, 8, 9, 62, 419, 8, 1926, 2, 1, 356, 4, 1, 173, 67, 51, 48, 17, 9, 78, 24, 5, 131, 112, 118, 355, 3, 113, 52, 17, 137, 3, 388, 8, 96, 25, 820, 7, 1575, 6, 40, 3, 515, 33, 211, 1277, 2, 1874, 7, 27, 310, 5, 603, 23, 109, 1, 16, 6, 7, 53, 92, 723, 34, 1, 273, 140, 25, 3, 100, 41, 39, 155, 698, 515, 1947, 16, 69, 21, 22, 204, 46, 6, 72, 49, 5, 8, 312, 140, 189, 3, 164, 29, 10, 16] 121


In [81]:
x_train = sequence.pad_sequences(x_train_seq,maxlen=200)
x_test = sequence.pad_sequences(x_test_seq,maxlen=200)

In [82]:
print(x_train[0],len(x_train[0]))

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    9  430    5   63   10
   16   15    3  756    4  365   27  310    9  156   62  838   72   40
    8   34    9  281 1012  228   17   99    9  215    8    9   62  419
    8 1926    2    1  356    4    1  173   67   51   48   17    9   78
   24    5  131  112  118  355    3  113   52   17  137    3  388    8
   96   25  820    7 1575    6   40    3  515   33  211 1277    2 1874
    7   27  310    5  603   23  109    1   16    6    7   53   92  723
   34    1  273  140   25    3  100   41   39  155  698  515 1947   16
   69   21   22  204   46    6   72   49    5    8  312  140  189    3
  164 

In [83]:
print(x_train_seq[1],len(x_train_seq[1]))

[9, 62, 506, 10, 16, 8, 558, 3, 180, 460, 61, 11, 4, 2, 8, 7, 3, 395, 158, 2, 677, 5, 1, 92, 1, 1719, 833, 1, 51, 271, 4, 677, 2, 291, 461, 27, 1, 507, 717, 42, 1, 2, 208, 4, 1, 293, 2, 21, 24, 5, 115, 13, 1, 3, 169, 59, 148, 1772, 7, 1, 293, 17, 62, 1518, 5, 179, 52, 1, 273, 1401, 42, 787, 113, 203, 106, 14, 41, 2] 78


In [84]:
print(x_train[1],len(x_train[1]))

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    9   62  506   10
   16    8  558    3  180  460   61   11    4    2    8    7    3  395
  158    2  677    5    1   92    1 1719  833    1   51  271    4  677
    2  291  461   27    1  507  717   42    1    2  208    4    1  293
    2   21   24    5  115   13    1    3  169   59  148 1772    7    1
  293   17   62 1518    5  179   52    1  273 1401   42  787  113  203
  106 

# Multilayers

In [45]:
from keras.models import Sequential
from keras.layers.core import Dense,Dropout,Activation,Flatten
from keras.layers.embeddings import Embedding

In [85]:
model = Sequential()

In [86]:
model.add(Embedding(output_dim=32,input_dim=4000,input_length=200))

In [87]:
model.add(Dropout(0.2))

In [88]:
model.add(Flatten())

In [89]:
model.add(Dense(units=256,activation='relu'))
model.add(Dropout(0.35))

model.add(Dense(units = 1,activation='sigmoid'))

In [90]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 200, 32)           128000    
_________________________________________________________________
dropout_3 (Dropout)          (None, 200, 32)           0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 6400)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               1638656   
_________________________________________________________________
dropout_4 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 257       
Total params: 1,766,913
Trainable params: 1,766,913
Non-trainable params: 0
_________________________________________________________________


In [91]:
model.compile(loss = 'binary_crossentropy',optimizer = 'adam',metrics = ['accuracy'])

In [92]:
train_history = model.fit(x_train,y_train,batch_size=100,epochs = 10,verbose = 1,validation_split=0.2)

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 7s 338us/step - loss: 0.4598 - acc: 0.7707 - val_loss: 0.5711 - val_acc: 0.7254
Epoch 2/10
20000/20000 [==============================] - 5s 231us/step - loss: 0.2304 - acc: 0.9079 - val_loss: 0.4656 - val_acc: 0.7954
Epoch 3/10
20000/20000 [==============================] - 5s 226us/step - loss: 0.1200 - acc: 0.9581 - val_loss: 0.5514 - val_acc: 0.8002
Epoch 4/10
20000/20000 [==============================] - 5s 231us/step - loss: 0.0578 - acc: 0.9811 - val_loss: 0.6160 - val_acc: 0.8116
Epoch 5/10
20000/20000 [==============================] - 5s 231us/step - loss: 0.0332 - acc: 0.9897 - val_loss: 0.9918 - val_acc: 0.7540
Epoch 6/10
20000/20000 [==============================] - 5s 230us/step - loss: 0.0274 - acc: 0.9903 - val_loss: 1.1355 - val_acc: 0.7430
Epoch 7/10
20000/20000 [==============================] - 5s 231us/step - loss: 0.0246 - acc: 0.9918 - val_loss: 0.9533 - v

In [93]:
scores = model.evaluate(x_test,y_test,verbose=1)

25000/25000 [==============================] - 1s 42us/step


In [94]:
scores

[0.8537006928491593, 0.8324]

In [56]:
predict = model.predict_classes(x_test,verbose=1)

25000/25000 [==============================] - 1s 25us/step


In [58]:
predict[-10:]

array([[1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0]], dtype=int32)

In [59]:
predict = predict.reshape(-1)

In [69]:
predict[12502]
y_test[12502]

0

In [70]:
SentimentDict = {1:'positive',0:'negative'}
def display_text_Sentiment(i):
    print(test_text[i])
    print('label True:',SentimentDict[y_test[i]],
          'Predict result:',SentimentDict[predict[i]])

In [71]:
display_text_Sentiment(1)

This is a gem of a film, Slapstick violence a-plenty, Simon pegg and Green Spew.A suspicious looking power plant and an incontrollable motorbike add to this stonker. If you haven't seen this I can highly recommend it, maybe not quite as good as bottom but not far short.If you have never seen a man impaled by the nose on two hooks then what are you waiting for buy this film.I didn't even see the twist at the end coming when it turns out that Ritchie and Eddie are both ghosts.Feeb - 1 boiled egg.
label True: positive Predict result: positive


In [72]:
display_text_Sentiment(12502)

How LIVING THE DREAM managed to get into the Laemmle 5 in West Hollywood is beyond me, as it is the worst film I have ever seen in my life. I should have known when the first scene opened in-gasp, Eugene, Oregon-,that this dud of a film with characters that you want to like and feel sorry for from their exclusion days from high school,but can't, as they are such losers, is so wooden and atrocious with dialog that is beyond bad.Then, cliché, the three high school losers end up in LA, and here is where the film could have been realistic if it had shown them trying to find a career in acting. But no, one works as a used car salesman, the other is a true loser in a garage call center selling magazines. Even the bastard that runs the place has more audience appeal than that dreadful actor with the horrible foreign accent. And, they fraudulently get money from an insurance scam to set up an Executive Recruitment firm with no experience, just showing "the supposed good life" in LA night spots

In [95]:
model.save_weights("IMDB-Multilayers(dict4000).h5")

In [98]:
def predict_review(input_text):
    input_seq = token.texts_to_sequences([input_text])
    pad_input_seq = sequence.pad_sequences(input_seq,maxlen=200)
    predict_result = model.predict_classes(pad_input_seq)
    print(SentimentDict[predict_result[0][0]])

In [100]:
text1 = 'i hate it'
predict_review(text1)

negative


In [101]:
text1 = 'i love it'
predict_review(text1)

positive
